In [1]:
import os
import csv
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
chunk_size = 200
collection_name = f"mycollection_{chunk_size}"
embedding_model = "text-embedding-3-large"
temperature = 0

In [3]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

In [4]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip"} 
    )
    print(f"コレクション {collection_name} のインスタンスを作成しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")

現在のコレクション一覧:
- mycollection_500
- mycollection_100
- mycollection_1000
- mycollection_200
コレクション mycollection_200 のインスタンスを作成しました


In [5]:
result = vector_store.get()

# 結果の表示
for i in range(min(5, len(result["ids"]))):
    print(f"\n--- ドキュメント {i+1} ---")
    print(f"ID: {result['ids'][i]} ソース: {result['metadatas'][i]['source']} 文字数: {len(result['documents'][i])}")
    print(f"{result['documents'][i]}")


--- ドキュメント 1 ---
ID: 52087784-a618-4da1-9c3b-b0efc470ce32 ソース: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf 文字数: 198
ハリー‧ポッター (架空の⼈物)出典: フリー百科事典『ウィキペディア（Wikipedia）』ハリー‧ポッターシリーズ > ハリー‧ポッターシリーズの登場⼈物⼀覧 > ハリー‧ポッター (架空の⼈物)ハリー‧ジェームズ‧ポッター（英: Harry JamesPotter）は、J‧K‧ローリングの⼩説『ハリー‧ポッター』シリーズおよび、その派⽣作品に登場する架空の⼈物であり、同シリーズの主⼈公。

--- ドキュメント 2 ---
ID: 74fa4331-e2fe-490b-b966-ea50c4f54502 ソース: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf 文字数: 169
ホグワーツ魔法魔術学校グリフィンドール寮の男⼦⽣徒となる。孤児として⺟親の親類の伯⺟夫婦の家で不遇な暮らしをして育った。11歳を迎える年のある⽇突然、ホグワーツから⼊学許可証が届いたのをきっかけに、亡くなった両親が魔法使いであったこと、そして出⽣時に下された予⾔により、闇の魔法使いヴォルデモートを倒す宿命を⾃分が負っていると告げられる。

--- ドキュメント 3 ---
ID: 0bb78ff4-cf20-406b-a458-09bf48e54c28 ソース: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf 文字数: 196
マグル界では⼀介の少年に過ぎない⽣活を送っていたが、魔法界では本⼈が⼾惑うほど重要な⼈物として、あまねく⼈々から知られている。⼀⼈前の魔法使いになるべく、同級⽣のロン‧ウィーズリーやハーマイオニー‧グレンジャーらとともに、ホグワーツにて学⽣⽣活を送りつつ、宿敵のヴォルデモートなどの闇の魔法使いたちによる数々の陰謀に⽴ち向かう冒険の⽇々を通して、たくましく成⻑していく姿が物語で描かれている。

--- ドキュメント 4 ---
ID: 5b82b2cb-09e9-420b-9469-97fbdcbca1c0 ソース: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf 文字数: 15

In [6]:
# ./tmp/ ディレクトリが存在しない場合は作成
os.makedirs('./tmp', exist_ok=True)

# CSVファイルに書き込み
csv_path = f'./tmp/{collection_name}.csv'
row_count = 0

with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # ヘッダーを書き込み
    writer.writerow(['ID', 'ソース', '文字数', 'コンテンツ'])
    
    # データを書き込み
    for i in range(len(result["ids"])):
        writer.writerow([
            result['ids'][i],
            result['metadatas'][i]['source'],
            len(result['documents'][i]),
            result['documents'][i]
        ])
        row_count += 1

print(f"データを {csv_path} に保存しました。")
print(f"合計 {row_count} 行のデータを書き込みました。")

データを ./tmp/mycollection_200.csv に保存しました。
合計 3378 行のデータを書き込みました。
